# Training and A/B test for Conversion agents - <font color='blue'> Proportion</font> of sale

In [1]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended 

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [2]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [3]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000 
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [4]:
# # Choose number of A/B tests
# num_AB_tests = 2 ##tochange !
num_AB_tests = 25 ##tochange !

#### Logs

In [5]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)

try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    from recogym.agents import RandomAgent, random_args
    random_agent = RandomAgent(Configuration(random_args))
    agents[name_agent] = random_agent
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Proportion</font> of sale

##### No weights

In [6]:
name_extension = 'prop'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, repo = data_repo)

--- Trained agents loaded --- 


In [7]:
try:
    for i in range(num_AB_tests):
        name_extension = 'prop'+'_nb'+str(i)
        name = name = 'likelihood_saleclickprod_discount_allprop_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'prop'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [8]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Click,0.642 (0.081) %,0 (0),10.06 (0.00) %,5 (2),30.98 (12.68) %
1,PDS,0.792 (0.032) %,1 (0),19.02 (4.23) %,5 (1),27.99 (0.00) %
2,PCS,0.521 (0.216) %,1 (0),13.04 (4.22) %,7 (2),30.98 (12.68) %
3,DPCS,0.816 (0.099) %,1 (0),16.03 (8.45) %,4 (3),22.01 (8.46) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Click,0.568 (0.012) %,5.06 (0.48) %,4.24 (0.83) %,4.13 (0.77) %,0.575 (0.042) pm,0.094 (0.041) pm,0.106 (0.035) pm
1,PDS,0.573 (0.081) %,6.34 (0.27) %,5.40 (0.21) %,5.74 (0.43) %,0.729 (0.134) pm,0.108 (0.022) pm,0.068 (0.008) pm
2,PCS,0.543 (0.103) %,4.82 (0.88) %,1.83 (0.41) %,3.92 (0.89) %,0.514 (0.003) pm,0.311 (0.079) pm,0.098 (0.019) pm
3,DPCS,0.656 (0.108) %,4.75 (0.57) %,3.40 (1.40) %,3.75 (0.89) %,0.629 (0.178) pm,0.168 (0.080) pm,0.127 (0.020) pm


In [9]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
 Click &  0.642 (0.081) \% &     0 (0) &  10.06 (0.00) \% &  5 (2) &  30.98 (12.68) \% \\
   PDS &  0.792 (0.032) \% &     1 (0) &  19.02 (4.23) \% &  5 (1) &   27.99 (0.00) \% \\
   PCS &  0.521 (0.216) \% &     1 (0) &  13.04 (4.22) \% &  7 (2) &  30.98 (12.68) \% \\
  DPCS &  0.816 (0.099) \% &     1 (0) &  16.03 (8.45) \% &  4 (3) &   22.01 (8.46) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
 Click &  0.568 (0.012) \% &  5.06 (0.48) \% &  4.24 (0.83) \% &  4.13 (0.77) \% &  0.575 (0.042) pm &  0.094 (0.041) pm &  0.106 (0.035) pm \\
   PDS &  0.573 (0.081) \% &  6.34 (0.27) \% &  5.40 (0.21) \% &  5.74 (0.43) \% &  0.729 (0.134) pm &  0.108 (0.022) pm &  0.068 (0.008) pm \\
   PCS &  0

##### Weights

In [10]:
name_extension = 'propweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, weights = True, repo = data_repo)

--- Trained agents loaded --- 


In [11]:
try:
    for i in range(num_AB_tests):
        name_extension = 'propweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpropweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'propweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [12]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Click,1.111 (0.854) %,1 (0),16.03 (8.45) %,4 (1),30.98 (4.23) %
1,PDS,0.642 (0.236) %,1 (0),19.02 (4.23) %,10 (1),33.97 (0.00) %
2,PCS,0.575 (0.150) %,1 (1),16.03 (8.45) %,7 (6),27.98 (16.91) %
3,DPCS,0.639 (0.268) %,1 (1),16.03 (8.45) %,8 (2),30.98 (12.68) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Click,0.613 (0.031) %,4.50 (0.94) %,3.01 (1.60) %,3.44 (1.27) %,0.554 (0.143) pm,0.180 (0.071) pm,0.129 (0.033) pm
1,PDS,0.530 (0.122) %,6.46 (0.56) %,5.42 (1.42) %,5.95 (0.84) %,0.678 (0.098) pm,0.121 (0.116) pm,0.057 (0.042) pm
2,PCS,0.665 (0.131) %,4.44 (0.98) %,3.64 (0.91) %,3.74 (0.95) %,0.578 (0.014) pm,0.105 (0.011) pm,0.094 (0.015) pm
3,DPCS,0.760 (0.091) %,4.61 (0.50) %,2.76 (0.87) %,3.60 (0.51) %,0.697 (0.007) pm,0.285 (0.090) pm,0.155 (0.020) pm


In [13]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &   Sales &               CR \\
\midrule
 Click &  1.111 (0.854) \% &     1 (0) &  16.03 (8.45) \% &   4 (1) &   30.98 (4.23) \% \\
   PDS &  0.642 (0.236) \% &     1 (0) &  19.02 (4.23) \% &  10 (1) &   33.97 (0.00) \% \\
   PCS &  0.575 (0.150) \% &     1 (1) &  16.03 (8.45) \% &   7 (6) &  27.98 (16.91) \% \\
  DPCS &  0.639 (0.268) \% &     1 (1) &  16.03 (8.45) \% &   8 (2) &  30.98 (12.68) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
 Click &  0.613 (0.031) \% &  4.50 (0.94) \% &  3.01 (1.60) \% &  3.44 (1.27) \% &  0.554 (0.143) pm &  0.180 (0.071) pm &  0.129 (0.033) pm \\
   PDS &  0.530 (0.122) \% &  6.46 (0.56) \% &  5.42 (1.42) \% &  5.95 (0.84) \% &  0.678 (0.098) pm &  0.121 (0.116) pm &  0.057 (0.042) pm \\
   PCS